<center> 
    <h1> MBTI Personality Trait Classification using Twitter </h1>
    <h4> <i> Author: Pavan Kumar K. N., Marina L. Gavrilova </i>  </h4>
 </center>


In [1]:
#
# * File:    classic_persona.py
# *
# * Author1:  Pavan Kumar K N (pavankumar.karkekopp@ucalgary.ca)
# * Date:     20th May 2019
# * Summary of File:
# * Explore mbti_1.csv file acquired from https://www.kaggle.com/datasnaek/mbti-type
# * Apply state-of-the-art reported publicly
# * Build classifier model that is better using machine learning techniques

#Just making sure the right environment is running this script
import sys
sys.executable


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.initializers import Constant
from keras import optimizers
import re
import os 
import nltk

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
mbti_data = pd.read_csv('data/mbti_1.csv', delimiter=',')

In [4]:
mbti_data = mbti_data[['posts', 'type']]

pd.set_option('display.max_colwidth', -1)
mbti_data.head(3)

posts  \
0  'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-home-design.jpg ...|||Welcome and stuff.|||http://playeressence.com/wp-content/uploads/2013/08/RED-red-the-pokemon-master-32560474-450-338.jpg  Game. Set. Match.|||Prozac, wellbrutin, at least thirty minutes of moving your legs (and I don't mean moving them while sitting in your same desk chair), weed in moderation (maybe try edibles as a healthier alternative...|||Basically come up with three items you've determined that each type (or whichever types you want to do) would more than likely use, given each types' cognitive functions and whatnot, when left by...|||All things in moderation.  Sims is indeed a video game, and a good one at that. Note: a good one at that is somewhat subjective in that I am not completely promoting the death of any given Sim...|||Dear ENFP:  What were your favorite video games growing up and what are your now, current favorite video games? :cool:|||https://www.youtube.com/watch?v=QyPqT8umzmY|||It appears to be too late. :sad:|||There's someone out there for everyone.|||Wait... I thought confidence was a good thing.|||I just cherish the time of solitude b/c i revel within my inner world more whereas most other time i'd be workin... just enjoy the me time while you can. Don't worry, people will always be around to...|||Yo entp ladies... if you're into a complimentary personality,well, hey.|||... when your main social outlet is xbox live conversations and even then you verbally fatigue quickly.|||http://www.youtube.com/watch?v=gDhy7rdfm14  I really dig the part from 1:46 to 2:50|||http://www.youtube.com/watch?v=msqXffgh7b8|||Banned because this thread requires it of me.|||Get high in backyard, roast and eat marshmellows in backyard while conversing over something intellectual, followed by massages and kisses.|||http://www.youtube.com/watch?v=Mw7eoU3BMbE|||http://www.youtube.com/watch?v=4V2uYORhQOk|||http://www.youtube.com/watch?v=SlVmgFQQ0TI|||Banned for too many b's in that sentence. How could you! Think of the B!|||Banned for watching movies in the corner with the dunces.|||Banned because Health class clearly taught you nothing about peer pressure.|||Banned for a whole host of reasons!|||http://www.youtube.com/watch?v=IRcrv41hgz4|||1) Two baby deer on left and right munching on a beetle in the middle.  2) Using their own blood, two cavemen diary today's latest happenings on their designated cave diary wall.  3) I see it as...|||a pokemon world  an infj society  everyone becomes an optimist|||49142|||http://www.youtube.com/watch?v=ZRCEq_JFeFM|||http://discovermagazine.com/2012/jul-aug/20-things-you-didnt-know-about-deserts/desert.jpg|||http://oyster.ignimgs.com/mediawiki/apis.ign.com/pokemon-silver-version/d/dd/Ditto.gif|||http://www.serebii.net/potw-dp/Scizor.jpg|||Not all artists are artists because they draw. It's the idea that counts in forming something of your own... like a signature.|||Welcome to the robot ranks, person who downed my self-esteem cuz I'm not an avid signature artist like herself. :proud:|||Banned for taking

In [5]:
raw_posts = mbti_data.posts.values

In [6]:
filtered_posts = [p.split("|||") for p in raw_posts]

In [7]:
filtered_posts[0]

["'http://www.youtube.com/watch?v=qsXHcwe3krw",
 'http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg',
 'enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks',
 'What has been the most life-changing experience in your life?',
 'http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.',
 'May the PerC Experience immerse you.',
 'The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206',
 "Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...",
 '84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-ho

In [8]:
#Function to binarize the types into simple lists instead of pandas.series
personality_binary = {'I':1, 'E':0, 'N':1,'S':0, 'T':1, 'F':0, 'J':1, 'P': 0}
binary_personality = [{1:'I', 0:'E'}, 
                      {1:'N', 0:'S'},
                      {1:'T', 0:'F'},
                      {1:'J', 0:'P'}]

def translate_personality(personality):
    # transform mbti to binary vector
    binary = [personality_binary[l] for l in personality]
#     print("{} returned {}".format(personality, binary))
    return binary


def translate_binary(personality):
    # transform binary vector to mbti personality
    s = ""
    for i, l in enumerate(personality):
        s += binary_personality[i][l]
#     print("{} returned {}".format(personality, s))
    return s

In [9]:
list_personality_bin = np.array([translate_personality(p) for p in mbti_data.type])
print("Binarize MBTI list: \n%s" % list_personality_bin)

Binarize MBTI list: 
[[1 1 0 1]
 [0 1 1 0]
 [1 1 1 0]
 ...
 [1 1 1 0]
 [1 1 0 0]
 [1 1 0 0]]


In [10]:
#List of strings to remove from the corpus
unique_type_list = ['INFJ', 
                    'ENTP', 
                    'INTP', 
                    'INTJ', 
                    'ENTJ', 
                    'ENFJ', 
                    'INFP', 
                    'ENFP',
                    'ISFP', 
                    'ISTP', 
                    'ISFJ', 
                    'ISTJ', 
                    'ESTP', 
                    'ESFP', 
                    'ESTJ', 
                    'ESFJ']

unique_type_list = [x.lower() for x in unique_type_list]

# Initialize for Lemmatization
stemmer = nltk.PorterStemmer()
lemmatiser = nltk.WordNetLemmatizer()

def pre_process_data(data, remove_stop_words = False, remove_mbti_profiles = True):
    list_personality = []
    list_posts = []
    len_data = len(data)
    i=0
    
    for row in data.iterrows():
        i+=1
        if (i % 500 == 0 or i == 1 or i == len_data):
            print("%s of %s rows" % (i, len_data))

        ##### Remove and clean comments
        posts = row[1].posts
        temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', posts)
#         temp = re.sub("[^a-zA-Z]", " ", temp)
        temp = re.sub(' +', ' ', temp).lower()
        if remove_stop_words:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in stopwords.words("english")])
#         else:
#             temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ')])
            
        if remove_mbti_profiles:
            for t in unique_type_list:
                temp = temp.replace(t,"")

        word_count = len(temp.split(' '))
        
        #Filter for minimum 50 words and max 2000 words
        if(word_count>50 and word_count<2000):
            type_labelized = translate_personality(row[1].type)
            list_personality.append(type_labelized)
            list_posts.append(temp)

    list_posts = np.array(list_posts)
    list_personality = np.array(list_personality)
    return list_posts, list_personality

In [11]:
list_posts, list_personality  = pre_process_data(mbti_data, remove_stop_words=False)

1 of 8675 rows
500 of 8675 rows
1000 of 8675 rows
1500 of 8675 rows
2000 of 8675 rows
2500 of 8675 rows
3000 of 8675 rows
3500 of 8675 rows
4000 of 8675 rows
4500 of 8675 rows
5000 of 8675 rows
5500 of 8675 rows
6000 of 8675 rows
6500 of 8675 rows
7000 of 8675 rows
7500 of 8675 rows
8000 of 8675 rows
8500 of 8675 rows
8675 of 8675 rows


In [12]:
word_counts = [len(str(x).split(' ')) for x in list_posts]

In [13]:
sequence_length = max(word_counts)

In [14]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', filters=' ')
tokenizer.fit_on_texts(list_posts)

In [15]:
# this takes our sentences and replaces each word with an integer
X = tokenizer.texts_to_sequences(list_posts)
X = pad_sequences(X, sequence_length)
y = list_personality
X_train = X
y_train = y
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
# print("test set size " + str(len(X_test)))

In [16]:
X

array([[    0,     0,     0, ...,   135,    15,     1],
       [    0,     0,     0, ...,     2,    16,  9410],
       [    0,     0,     0, ...,     3,  1207,     1],
       ...,
       [    0,     0,     0, ...,     1,   432,     1],
       [    0,     0,     0, ...,    16,  1362,     1],
       [    0,     0,     0, ...,    86,    15, 14568]])

In [17]:
embeddings_index = {}
f = open(os.path.join('data/glove.twitter.27B/', 'glove.twitter.27B.100d.txt'), encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [18]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 526155 unique tokens.


In [19]:
num_words = min(max_features, len(word_index)) + 1
print(num_words)

embedding_dim = 100
not_exist_count = 0
# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
#     else:
#         # doesn't exist, assign a random vector
#         embedding_matrix[i] = np.random.randn(embedding_dim)
#         not_exist_count+= 1
# print(not_exist_count)

20001


In [20]:
# #Stacked LSTMs
# model = Sequential()
# model.add(Embedding(num_words,
#                     embedding_dim,
#                     embeddings_initializer=Constant(embedding_matrix),
#                     input_length=sequence_length,
#                     trainable=True))

# model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
# model.add(Dropout(0.2))
# model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
# model.add(Dropout(0.2))
# model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
# model.add(Dropout(0.2))
# model.add(Bidirectional(CuDNNLSTM(128, return_sequences=False)))
# model.add(Dropout(0.2))
# # model.add(Flatten())
# model.add(Dense(units=4, activation='softmax'))
# sgd= optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss = 'categorical_crossentropy', optimizer=sgd, metrics = ['accuracy'])
# print(model.summary())

In [21]:
model = Sequential()
model.add(Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=sequence_length,
                    trainable=True))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Conv1D(128, 16, activation='relu'))
model.add(MaxPooling1D(25))
model.add(Flatten())
model.add(Dense(units=4, activation='softmax'))
sgd= optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss = 'categorical_crossentropy', optimizer=sgd, metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1881, 100)         2000100   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1881, 256)         235520    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1881, 256)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1881, 256)         395264    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1881, 256)         0         
_________________________________________________________________
bidirectional_3 (Bidi

In [22]:
batch_size = 64
history = model.fit(X_train, y_train, epochs=100, batch_size=batch_size, verbose=1, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 6938 samples, validate on 1735 samples
Epoch 1/100
6938/6938 [==============================] - 112s 16ms/step - loss: 3.3238 - acc: 0.2619 - val_loss: 3.3504 - val_acc: 0.2156
Epoch 2/100
6938/6938 [==============================] - 109s 16ms/step - loss: 3.3137 - acc: 0.2577 - val_loss: 3.3593 - val_acc: 0.2277
Epoch 3/100
6938/6938 [==============================] - 109s 16ms/step - loss: 3.3115 - acc: 0.2571 - val_loss: 3.3630 - val_acc: 0.2438
Epoch 4/100
6938/6938 [==============================] - 109s 16ms/step - loss: 3.3065 - acc: 0.2638 - val_loss: 3.3438 - val_acc: 0.4213
Epoch 5/100
6938/6938 [==============================] - 110s 16ms/step - loss: 3.3029 - acc: 0.2499 - val_loss: 3.3279 - val_acc: 0.4548
Epoch 6/100
6938/6938 [==============================] - 110s 16ms/step - loss: 3.2999 - acc: 0.2580 - val_loss: 3.3519 - val_acc: 0.2386
Epoch 7/100
6938/6938 [==============================] - 110s 16ms/step - lo

Epoch 58/100
6938/6938 [==============================] - 112s 16ms/step - loss: 3.1461 - acc: 0.4275 - val_loss: 3.3378 - val_acc: 0.2767
Epoch 59/100
6938/6938 [==============================] - 111s 16ms/step - loss: 3.1431 - acc: 0.4346 - val_loss: 3.3151 - val_acc: 0.3274
Epoch 60/100
6938/6938 [==============================] - 111s 16ms/step - loss: 3.1372 - acc: 0.4385 - val_loss: 3.3169 - val_acc: 0.3470
Epoch 61/100
6938/6938 [==============================] - 111s 16ms/step - loss: 3.1353 - acc: 0.4295 - val_loss: 3.3109 - val_acc: 0.3591
Epoch 62/100
6938/6938 [==============================] - 112s 16ms/step - loss: 3.1302 - acc: 0.4363 - val_loss: 3.3128 - val_acc: 0.4023
Epoch 63/100
6938/6938 [==============================] - 112s 16ms/step - loss: 3.1250 - acc: 0.4403 - val_loss: 3.3308 - val_acc: 0.4784
Epoch 64/100
6938/6938 [==============================] - 112s 16ms/step - loss: 3.1201 - acc: 0.4396 - val_loss: 3.3136 - val_acc: 0.4795
Epoch 65/100
6938/6938 [===

In [23]:
X_train

array([[    0,     0,     0, ...,   135,    15,     1],
       [    0,     0,     0, ...,     2,    16,  9410],
       [    0,     0,     0, ...,     3,  1207,     1],
       ...,
       [    0,     0,     0, ...,     1,   432,     1],
       [    0,     0,     0, ...,    16,  1362,     1],
       [    0,     0,     0, ...,    86,    15, 14568]])